In [0]:
%sql

CREATE OR REPLACE TABLE default.materialized_viw_requestinvoices AS
SELECT
  SUM(COALESCE(try_cast(ri.chargedamount as decimal(10, 2)), 0)) AS chargedamount,
  ri.foirequestid,
  ri.sourceoftruth
FROM
  factRequestInvoices ri
WHERE
  ri.activeflag = 'Y'
  -- AND ri.invoicenumber <> '0'
  AND ri.sourceoftruth = 'FOIMOD'
GROUP BY
  ri.foirequestid,
  ri.sourceoftruth;

In [0]:
%sql

CREATE OR REPLACE TABLE default.materialized_viw_activeoipcreviews AS
SELECT DISTINCT
  rd.visualrequestfilenumber,
  frccf.oipcno,
  rd.primaryusername,
  rd.startdate,
  rd.targetdate,
  frccf.reviewtype,
  frccf.customfieldstatus,
  frccf.reason,
  rd.requestdescription,
  frccf.subject,
  rqt.requestertypename,
  frccf.portfolioofficer,
  frccf.judicialreview,
  rd.status,
  rs.requeststatusname,
  rd.officeid AS procorg,
  rd.sourceoftruth
FROM
  -- default.factRequestDetails rd
    (
      SELECT
          *,
          ROW_NUMBER() OVER(PARTITION BY foirequestid ORDER BY runcycleid DESC) as rn
      FROM
          factRequestDetails
      WHERE
          sourceoftruth = 'FOIMOD' AND activeflag = 'Y'
    ) AS rd
    -- LEFT JOIN default.dimRequests r ON (rd.foirequestid = r.foirequestid AND rd.sourceoftruth = r.sourceoftruth)
    -- LEFT JOIN default.factRequestRequesters rr
    --   ON r.foirequestid = rr.foirequestid
    --   AND rr.activeflag = 'Y'
    --   AND rd.sourceoftruth = rr.sourceoftruth
    LEFT JOIN (
        SELECT
            *,
            ROW_NUMBER() OVER(PARTITION BY foirequestid ORDER BY runcycleid DESC) as rn
        FROM
            factRequestRequesters
        WHERE
            sourceoftruth = 'FOIMOD' AND activeflag = 't'
    ) AS rr ON rd.foirequestid = rr.foirequestid AND rr.rn = 1
    LEFT JOIN default.dimRequesterTypes rqt
      ON rr.requestertypeid = rqt.requestertypeid AND rr.sourceoftruth = rqt.sourceoftruth
    LEFT JOIN default.dimRequestStatuses rs
      ON rd.requeststatusid = rs.requeststatusid AND rd.sourceoftruth = rs.sourceoftruth
    -- LEFT JOIN default.dimECOffice eco
    --   ON rd.sourceoftruth != 'FOIMOD' AND try_cast(rd.officeid AS BIGINT) = eco.officeid
    -- LEFT JOIN default.factRequestCustomCalcFields frccf
    --   ON rd.foirequestid = frccf.foirequestid AND rd.sourceoftruth = frccf.sourceoftruth
    LEFT JOIN (
        SELECT
            *,
            ROW_NUMBER() OVER(PARTITION BY foirequestid ORDER BY runcycleid DESC) as rn
        FROM
            factRequestCustomCalcFields
        WHERE
            sourceoftruth = 'FOIMOD'
    ) AS frccf ON rd.foirequestid = frccf.foirequestid AND frccf.rn = 1
WHERE
  -- rd.requesttypeid = (
  --   SELECT
  --     rt.requesttypeid
  --   FROM
  --     dimRequestTypes rt
  --   WHERE
  --     rt.requesttypename = 'Review'
  -- )
  rd.oipcno is not null
  AND rd.rn = 1;

In [0]:
%sql

CREATE OR REPLACE TABLE default.materialized_viw_additionalfields AS
SELECT DISTINCT
  rd.visualrequestfilenumber as requestid,
  CASE
    WHEN frccf.identityverification = 'NULL' THEN null
    ELSE frccf.identityverification
  END AS identityverification,
  CASE
    WHEN frccf.previousfoirequest = 'NULL' THEN null
    ELSE frccf.previousfoirequest
  END AS previousfoirequest,
  CASE
    WHEN frccf.physicalpageestimate = 'NULL' THEN null
    ELSE frccf.physicalpageestimate
  END AS physicalpageestimate,
  CASE
    WHEN frccf.electronicpageestimate = 'NULL' THEN null
    ELSE frccf.electronicpageestimate
  END AS electronicpageestimate,
  CASE
    WHEN frccf.noofpagesdeduplicated = 'NULL' THEN null
    ELSE frccf.noofpagesdeduplicated
  END AS noofpagesdeduplicated,
  CASE
    WHEN frccf.applicationfeepaid = 'NULL' THEN null
    ELSE frccf.applicationfeepaid
  END AS applicationfeepaid,
  CASE
    WHEN frccf.applicationfeerefunded = 'NULL' THEN null
    ELSE frccf.applicationfeerefunded
  END AS applicationfeerefunded,
  CASE
    WHEN frccf.applicationfeetransferred = 'NULL' THEN null
    ELSE frccf.applicationfeetransferred
  END AS applicationfeetransferred,
  CASE
    WHEN frccf.linkedrequests = 'NULL' THEN null
    ELSE frccf.linkedrequests
  END AS linkedrequests,
  rd.officeid AS actionoffice,
  rd.closeddate AS closedate,
  rd.targetdate AS duedate,
  try_cast(rd.foirequestid AS BIGINT) AS foirequestid,
  try_cast(rd.officeid AS BIGINT) AS officeid,
  rd.visualrequestfilenumber,
  rd.amendmentcreateddate,
  CASE
    WHEN rd.refvisualrequestfilenumber = 'NULL' THEN null
    ELSE rd.refvisualrequestfilenumber
  END AS refvisualrequestfilenumber,
  CASE
    WHEN rd.closedby = 'NULL' THEN null
    ELSE rd.closedby
  END AS closedby,
  rd.closeddate,
  rd.notes,
  rd.receivedbyusername,
  rd.createddate,
  rd.currentactivity,
  rd.deliverydate,
  rd.targetdate,
  rd.disposition,
  rd.originalcloseddate,
  rd.originaltargetdate,
  rd.originalreceiveddate,
  rd.primaryusername,
  try_cast(rd.processeddays AS BIGINT) AS processeddays,
  rd.receiveddate,
  CASE
    WHEN rd.referencenumber = 'NULL' THEN null
    ELSE rd.referencenumber
  END AS referencenumber,
  try_cast(rd.remainingdays AS BIGINT) AS remainingdays,
  rd.requestdescription,
  rd.caseowner,
  rd.status,
  rd.startdate,
  try_cast(rd.shipaddressid AS BIGINT) AS shipaddressid,
  try_cast(rd.billaddressid AS BIGINT) AS billaddressid,
  try_cast(rd.receivedmodeid AS BIGINT) AS receivedmodeid,
  try_cast(rd.deliverymodeid AS BIGINT) AS deliverymodeid,
  try_cast(rd.primarygroupid AS BIGINT) AS primarygroupid,
  try_cast(rd.requesttypeid AS BIGINT) AS requesttypeid,
  try_cast(rd.onbehalfofrequesterid AS BIGINT) AS onbehalfofrequesterid,
  rd.sourceoftruth
FROM
  -- factRequestDetails rd
    (
      SELECT
          *,
          ROW_NUMBER() OVER(PARTITION BY foirequestid ORDER BY runcycleid DESC) as rn
      FROM
          factRequestDetails
      WHERE
          sourceoftruth = 'FOIMOD' AND activeflag = 'Y'
    ) AS rd
    -- LEFT JOIN dimECOffice eco
    --   ON rd.sourceoftruth != 'FOIMOD' AND try_cast(rd.officeid AS BIGINT) = eco.officeid
    -- LEFT JOIN factRequestCustomCalcFields frccf
    --   ON rd.foirequestid = frccf.foirequestid AND rd.sourceoftruth = frccf.sourceoftruth
    LEFT JOIN (
        SELECT
            *,
            ROW_NUMBER() OVER(PARTITION BY foirequestid ORDER BY runcycleid DESC) as rn
        FROM
            factRequestCustomCalcFields
        WHERE
            sourceoftruth = 'FOIMOD'
    ) AS frccf ON rd.foirequestid = frccf.foirequestid AND frccf.rn = 1
WHERE
  rd.rn = 1
  -- and rd.sourceoftruth = 'FOIMOD';

In [0]:
%sql

-- currently join the most recent extension from foirequestextensions

CREATE OR REPLACE TABLE default.materialized_viw_broaderaxisfields AS
SELECT DISTINCT
  rd.visualrequestfilenumber,
  try_cast(re.extensiondays AS BIGINT) AS extensiondays,
  try_cast(frccf.noofdocdelivered AS BIGINT) AS noofdocdelivered,
  try_cast(frccf.noofpagesreleased AS BIGINT) AS noofpagesreleased,
  try_cast(frccf.noofpagesreviewed AS BIGINT) AS noofpagesreviewed,
  rd.actionoffice,
  rd.amendmentcreateddate,
  rpt.estimatedamount,
  rpt.paidamount,
  concat_ws(
    ', ', abill.address1, abill.address2, abill.city, abill.state, abill.country, abill.zipcode
  ) AS applicantbillingaddress,
  rqstrt.requestertypename,
  rqstr.email,
  rqstr.fax,
  rpt.applicationfee,
  rd.refvisualrequestfilenumber,
  rqstr.home,
  rqstr.mobile,
  rqstr.fullname,
  concat_ws(
    ', ', aship.address1, aship.address2, aship.city, aship.state, aship.country, aship.zipcode
  ) AS applicantshippingaddress,
  rqstr.workphone1,
  rqstr.workphone2,
  rpt.invoicedamount - rpt.paidamount AS balanceamount,
  rd.closedby,
  rd.closeddate,
  rd.notes,
  frccf.coordinatednrresponsereqd,
  rd.receivedbyusername,
  rd.createddate,
  rd.currentactivity,
  rd.deliverydate,
  dm.deliverymodename,
  rd.targetdate,
  re.approveddate,
  re.approvedcomments,
  re.completeddate,
  re.completedcomments,
  re.comments,
  et.extensiontypename,
  CASE
    WHEN
      re.completeddate IS NOT NULL AND re.approvedstatus = 'Approved'
    THEN
      'Completed (Approved)'
    WHEN
      re.completeddate IS NOT NULL AND re.approvedstatus = 'Denied'
    THEN
      'Completed (Denied)'
    ELSE re.approvedstatus
  END AS extensionstatus,
  rpt.feewaivedescription,
  rpt.feewaiverequested,
  rpt.feewaivegranted,
  rd.disposition,
  rpt.invoicedamount,
  rpt.invoicedate,
  rpt.invoicenumber,
  frccf.applicantfilereference,
  pm.paymentmodename,
  frccf.noofpagesdeduplicated,
  onbehalfofrequesterid.fullname AS onbehalfof,
  rqstr.company,
  rd.originalcloseddate,
  rd.originaltargetdate,
  rd.originalreceiveddate,
  rpt.paymentreceiveddate,
  rpt.paymentstatus,
  rd.primaryusername,
  ecg.groupname,
  try_cast(rd.processeddays AS BIGINT) AS processeddays,
  rd.receiveddate,
  rm.receivedmodename,
  rd.referencenumber,
  frccf.refundamount,
  try_cast(rd.remainingdays AS BIGINT) AS remainingdays,
  rd.requestdescription,
  rd.caseowner,
  rd.status,
  rt.requesttypename,
  frccf.secondaryusers,
  rd.startdate,
  frccf.subject,
  try_cast(rdd.noofpagesintherequest AS BIGINT) AS noofpagesintherequest,
  rdd.noofpagesinreviewlog,
  try_cast(frccf.onholddays AS BIGINT) AS onholddays,
  rpt.totalinvoicedamount,
  frccf.crossgovtno,
  COALESCE(ri.chargedamount, 0) AS chargedamount,
  rd.sourceoftruth
FROM
  materialized_viw_additionalfields rd
  LEFT JOIN dimRequests r ON rd.foirequestid = r.foirequestid AND rd.sourceoftruth = r.sourceoftruth AND r.cactive = 'Y'
  LEFT JOIN dimAddress aship ON aship.sourceoftruth = 'FOIMOD' AND aship.foirequestid = rd.foirequestid
  LEFT JOIN dimAddress abill ON abill.sourceoftruth = 'FOIMOD' AND abill.foirequestid = rd.foirequestid
  -- LEFT JOIN factRequestExtensions re ON rd.foirequestid = re.foirequestid AND re.activeflag = 'Y' AND rd.sourceoftruth = re.sourceoftruth
  LEFT JOIN (
      SELECT
          *,
          ROW_NUMBER() OVER(PARTITION BY foirequestid ORDER BY runcycleid DESC) as rn
      FROM
          factRequestExtensions
      WHERE
          sourceoftruth = 'FOIMOD' AND activeflag = in ('true', 't')
  ) AS re ON rd.foirequestid = re.foirequestid AND re.rn = 1
  LEFT JOIN dimExtensionTypes et ON re.extensiontypeid = et.extensiontypeid AND re.sourceoftruth = et.sourceoftruth
  -- LEFT JOIN factRequestRequesters rr ON rd.foirequestid = rr.foirequestid AND rr.activeflag = 'Y' AND rd.sourceoftruth = rr.sourceoftruth
  LEFT JOIN (
      SELECT
          *,
          ROW_NUMBER() OVER(PARTITION BY foirequestid ORDER BY runcycleid DESC) as rn
      FROM
          factRequestRequesters
      WHERE
          sourceoftruth = 'FOIMOD' AND activeflag = 'Y'
  ) AS rr ON rd.foirequestid = rr.foirequestid AND rr.rn = 1
  LEFT JOIN dimRequesters rqstr ON rqstr.sourceoftruth = rd.sourceoftruth AND rr.requesterid = rqstr.requesterid AND rqstr.cactive = 'Y'
  LEFT JOIN dimReceivedModes rm ON rd.receivedmodeid = rm.receivedmodeid AND rd.sourceoftruth = rm.sourceoftruth
  LEFT JOIN dimDeliveryModes dm ON rd.deliverymodeid = dm.deliverymodeid AND rd.sourceoftruth = dm.sourceoftruth
  LEFT JOIN dimECGroups ecg ON rd.primarygroupid = ecg.groupid AND rd.sourceoftruth = ecg.sourceoftruth
  LEFT JOIN dimRequestTypes rt ON rd.requesttypeid = rt.requesttypeid
  -- LEFT JOIN factRequestDocumentsDetails rdd ON rd.foirequestid = rdd.foirequestid AND rdd.activeflag = 'Y' AND rd.sourceoftruth = rdd.sourceoftruth
  LEFT JOIN (
      SELECT
          *,
          ROW_NUMBER() OVER(PARTITION BY foirequestid ORDER BY runcycleid DESC) as rn
      FROM
          factRequestDocumentsDetails
      WHERE
          sourceoftruth = 'FOIMOD' AND activeflag = 'Y'
  ) AS rdd ON rd.foirequestid = rdd.foirequestid AND rdd.rn = 1
  -- LEFT JOIN factRequestPaymentTransaction rpt ON rd.foirequestid = rpt.foirequestid AND rpt.activeflag = 'Y' AND rd.sourceoftruth = rpt.sourceoftruth
  LEFT JOIN (
      SELECT
          *,
          ROW_NUMBER() OVER(PARTITION BY foirequestid ORDER BY runcycleid DESC) as rn
      FROM
          factRequestPaymentTransaction
      WHERE
          sourceoftruth = 'FOIMOD' AND activeflag = 'Y'
  ) AS rpt ON rd.foirequestid = rpt.foirequestid AND rpt.rn = 1
  LEFT JOIN dimPaymentModes pm ON try_cast(rpt.paymentmodeid AS INT) = pm.paymentmodeid
  LEFT JOIN dimRequesters onbehalfofrequesterid
    ON onbehalfofrequesterid.sourceoftruth = 'FOIMOD'
    AND onbehalfofrequesterid.foirequestid = rd.foirequestid
    AND onbehalfofrequesterid.requesterid = try_cast(rd.onbehalfofrequesterid AS BIGINT)
  LEFT JOIN dimRequesterTypes rqstrt ON rr.requestertypeid = rqstrt.requestertypeid AND rr.sourceoftruth = rqstrt.sourceoftruth
  -- LEFT JOIN factRequestCustomCalcFields frccf ON rd.foirequestid = frccf.foirequestid AND rd.sourceoftruth = frccf.sourceoftruth
  LEFT JOIN (
      SELECT
          *,
          ROW_NUMBER() OVER(PARTITION BY foirequestid ORDER BY runcycleid DESC) as rn
      FROM
          factRequestCustomCalcFields
      WHERE
          sourceoftruth = 'FOIMOD'
  ) AS frccf ON rd.foirequestid = frccf.foirequestid AND frccf.rn = 1
  LEFT JOIN viw_requestinvoices ri ON rd.foirequestid = try_cast(ri.foirequestid AS BIGINT) AND rd.sourceoftruth = ri.sourceoftruth

In [0]:
%sql

-- MERGE INTO default.materialized_viw_allopenclosed AS targettable
-- USING (
CREATE OR REPLACE TABLE default.materialized_viw_allopenclosed AS
    SELECT
        *
    FROM (
        SELECT DISTINCT
            rd.visualrequestfilenumber,
            rd.groupname,
            rd.caseowner,
            CASE
                WHEN publicbody.name is NOT NULL THEN publicbody.name
                WHEN publicbodyold.name is NOT NULL THEN publicbodyold.name
                ELSE rd.ministry
            END AS ministry,
            -- rd.ministry,
            CASE
                WHEN publicbody.iaocode is NOT NULL THEN publicbody.iaocode
                WHEN eco.officecode is NOT NULL THEN eco.officecode
                ELSE rd.officeid
            END AS officecode,
            -- rd.officeid AS officecode,
            rt.requesttypename,
            category.requestertypename,
            CASE
                WHEN req.fullname is NOT NULL THEN req.fullname
                WHEN reqaxis.fullname is NOT NULL THEN reqaxis.fullname
                ELSE NULL
            END AS fullname,
            -- req.fullname,
            rd.subject,
            rd.startdate,
            rd.targetdate,
            CASE
                WHEN rd.requeststatus = 'Closed' THEN rd.closeddate
                ELSE NULL
            END AS closeddate,
            rd.processeddays,
            rd.currentactivity,
            -- rd.primaryusername,
            CASE
                WHEN rd.sourceoftruth = 'FOIMOD' THEN CONCAT_WS(', ', foiusers.lastname, foiusers.firstname)
                ELSE rd.primaryusername
            END AS primaryusername,
            rd.crossgovtno,
            rd.requestdescription,
            rs.requeststatusname,
            -- custom.countontime,
            -- custom.countoverdue,
            -- custom.onholddays,
            -- custom.daysoverdue,
            rd.countontime,
            rd.countoverdue,
            COALESCE(rd.onholddays, 0)+COALESCE(rd.onholdotherdays, 0) AS onholddays,
            -- rd.onholddays,
            rd.daysoverdue,
            pt.estimatedamount,
            pt.feewaived,
            custom.feepaidamount,
            rd.disposition,
            rd.publication,
            rd.publicationreason,
            ext.numberofextensions,
            CASE
                WHEN try_cast(frd.lanpagecount AS BIGINT) IS NULL OR try_cast(frd.lanpagecount AS BIGINT) = 0 THEN try_cast(rdd.noofpagesintherequest AS BIGINT)
                ELSE try_cast(frd.lanpagecount AS BIGINT)
            END AS noofpagesintherequest,
            try_cast(frd.recordspagecount AS BIGINT) AS recordspagecount,
            -- rd.isconsultflag,
            -- rd.isphasedrelease,
            -- rd.isoipcreview,
            CASE 
                WHEN rd.isconsultflag IS NULL OR LOWER(rd.isconsultflag) IN ('N', 'f', 'false', 'null') THEN 'N'
                WHEN LOWER(rd.isconsultflag) IN ('Y', 't', 'true') THEN 'Y'
                ELSE 'N'
            END AS isconsultflag,
            CASE 
                WHEN rd.isphasedrelease IS NULL OR LOWER(rd.isphasedrelease) IN ('N', 'f', 'false', 'null') THEN 'N'
                WHEN LOWER(rd.isphasedrelease) IN ('Y', 't', 'true') THEN 'Y'
                ELSE 'N'
            END AS isphasedrelease,
            CASE 
                WHEN rd.isoipcreview IS NULL OR LOWER(rd.isoipcreview) IN ('N', 'f', 'false', 'null') THEN 'N'
                WHEN LOWER(rd.isoipcreview) IN ('Y', 't', 'true') THEN 'Y'
                ELSE 'N'
            END AS isoipcreview,
            ROW_NUMBER() OVER(PARTITION BY rd.visualrequestfilenumber ORDER BY closeddate DESC) as rownum
        FROM
            (
            SELECT
                *,
                ROW_NUMBER() OVER(PARTITION BY visualrequestfilenumber ORDER BY runcycleid DESC) as rn
            FROM
                factRequestDetails
            WHERE
                activeflag = 'Y'
            ) AS rd
            LEFT JOIN dimRequests AS r ON (rd.foirequestid = r.foirequestid AND rd.sourceoftruth = r.sourceoftruth) AND r.cactive = 'Y'
            LEFT JOIN dimRequestTypes AS rt ON rt.requesttypeid = rd.requesttypeid AND rt.cactive = 'Y'
            -- LEFT JOIN factRequestDocumentsDetails AS rdd ON rdd.foirequestid = rd.foirequestid
            -- AND rdd.activeflag = 'Y'
            LEFT JOIN (
                SELECT
                    *,
                    ROW_NUMBER() OVER(PARTITION BY foirequestid, sourceoftruth ORDER BY runcycleid DESC) as rn
                FROM
                    factRequestDocumentsDetails
                WHERE
                    activeflag = 'Y'
            ) AS rdd ON rd.foirequestid = rdd.foirequestid AND rd.sourceoftruth = rdd.sourceoftruth AND rdd.rn = 1
            LEFT JOIN dimRequestStatuses AS rs ON rd.requeststatus = rs.requeststatus AND rd.sourceoftruth = rs.sourceoftruth
            LEFT JOIN dimRequesterTypes AS category ON rd.applicantcategoryid = category.requestertypeid AND rd.sourceoftruth = category.sourceoftruth
            LEFT JOIN dimRequesters AS req ON req.sourceoftruth = 'FOIMOD' AND req.foirequestid = rd.foirequestid AND req.requesterid = rd.requesterid
            LEFT JOIN dimRequesters AS reqaxis ON (reqaxis.sourceoftruth IS NULL OR reqaxis.sourceoftruth = 'AXIS') AND reqaxis.requesterid = rd.requesterid
            -- LEFT JOIN factRequestCustomCalcFields AS custom ON rd.foirequestid = custom.foirequestid
            LEFT JOIN (
                SELECT
                    *,
                    ROW_NUMBER() OVER(PARTITION BY foirequestid, sourceoftruth ORDER BY runcycleid DESC) as rn
                FROM
                    factRequestCustomCalcFields
            ) AS custom ON rd.foirequestid = custom.foirequestid AND rd.sourceoftruth = custom.sourceoftruth AND custom.rn = 1
            LEFT JOIN factRequestInvoices AS ri ON rd.foirequestid = ri.foirequestid AND ri.activeflag = 'Y' AND ri.sourceoftruth = rd.sourceoftruth
            LEFT JOIN dimMODMinistryRequestIDs AS rid ON rd.foirequestid = rid.foirequestid
            LEFT JOIN (
                SELECT
                    foirequestid,
                    sourceoftruth,
                    COUNT(DISTINCT requestextid) AS numberofextensions
                FROM
                    factRequestExtensions
                WHERE
                    approvedstatus in ('Approved', 'A')
                    AND activeflag in ('true', 't', 'Y')
                GROUP BY
                    foirequestid,
                    sourceoftruth
            ) AS ext ON rd.foirequestid = ext.foirequestid AND rd.sourceoftruth = ext.sourceoftruth
            LEFT JOIN (
                SELECT
                    foirequestnumber,
                    recordspagecount,
                    lanpagecount,
                    ROW_NUMBER() OVER(PARTITION BY foirequestnumber ORDER BY foiflowrequeststatusdetailsid DESC) as rn
                FROM
                    factFOIFlowRequestStatusDetails
                WHERE isactive = true
            ) AS frd ON frd.foirequestnumber = rd.visualrequestfilenumber AND frd.rn = 1
            -- LEFT JOIN (
            --     SELECT DISTINCT
            --         a.foirequestnumber,
            --         a.recordspagecount
            --     FROM
            --         factFOIFlowRequestStatusDetails AS a
            --         JOIN (
            --             SELECT
            --                 foirequestnumber,
            --                 MAX(version) AS maxv
            --             FROM
            --                 factFOIFlowRequestStatusDetails
            --             WHERE
            --                 recordspagecount IS NOT NULL
            --                 AND isactive = TRUE
            --             GROUP BY
            --                 foirequestnumber
            --         ) AS c ON c.foirequestnumber = a.foirequestnumber
            --         AND c.maxv = a.version
            --     WHERE
            --         a.recordspagecount IS NOT NULL
            --         AND a.isactive = TRUE
            -- ) AS frd ON frd.foirequestnumber = rd.visualrequestfilenumber
            LEFT JOIN factRequestPaymentTransaction AS pt ON rd.foirequestid = pt.foirequestid AND pt.activeflag = 'Y' AND pt.sourceoftruth = rd.sourceoftruth
            LEFT JOIN foi_mod.foiusers AS foiusers ON rd.primaryusername = foiusers.preferred_username AND (foiusers.isactive = 'true' OR foiusers.isactive = 't')
            LEFT JOIN foi_mod.programareas AS publicbody ON rd.officeid = publicbody.bcgovcode
            LEFT JOIN dimecoffice AS eco ON try_cast(rd.officeid AS BIGINT) = eco.officeid
            LEFT JOIN foi_mod.programareas AS publicbodyold ON eco.officecode = publicbodyold.iaocode
        WHERE
            rd.rn = 1
            AND (
                rd.closeddate >= TIMESTAMP('2014-01-01 00:00:00')
                OR rd.closeddate = TIMESTAMP('1900-01-01 00:00:00')
                OR rd.closeddate is Null
            )
            AND COALESCE(eco.officecode, 'NO CODE') NOT IN ('IAO', 'IMB', 'XGR', 'PROAC')
            AND COALESCE(publicbody.iaocode, 'NO CODE') NOT IN ('IAO', 'IMB', 'XGR', 'PROAC')
    ) AS temp
    WHERE temp.rownum =1



In [0]:
%sql

CREATE OR REPLACE TABLE default.materialized_viw_cfrrfdreport AS
SELECT
  rd.visualrequestfilenumber AS Request_ID,
  rd.redactiondescription AS Request_Description,
  rfd.createddate AS Request_Start_Date,
  eco.officedescription AS Office_of_Primary_Interest,
  rfd.completeddate AS Completed_Date,
  rfds.reqfordocstatus AS RFD_Status,
  rd.status AS Request_Status,
  rd.primaryusername AS Primary_User,
  rfd.requestdescription AS RFD_Comments,
  frfdcf.rfdage AS RFD_Age,
  rd.requestage AS Request_Age,
  rfd.duedate AS Due_Date,
  rfd.createddate AS Requested_Date,
  frfdcf.remainingdays AS Remaining_Days,
  frfdcf.elapseddays AS Processed_Days,
  rfd.requesttypeid AS Request_Type,
  eco.officedescription AS Office_Name,
  rd.officeid AS Office_Code,
  rfd.sourceoftruth
FROM
  -- factRequestForDocuments rfd
  (
      SELECT
          *,
          ROW_NUMBER() OVER(PARTITION BY foirequestid ORDER BY runcycleid DESC) as rn
      FROM
          factRequestForDocuments
      WHERE
          sourceoftruth = 'FOIMOD' AND activeflag = 'Y'
  ) AS rfd
  -- LEFT JOIN dimRequests r
  --   ON (
  --     rfd.foirequestid = r.foirequestid
  --     and r.sourceoftruth = rfd.sourceoftruth
  --   )
  -- LEFT JOIN factRequestDetails rd
  --   ON r.foirequestid = rd.foirequestid
  --   AND rd.activeflag = 'Y'
  LEFT JOIN (
      SELECT
          *,
          ROW_NUMBER() OVER(PARTITION BY foirequestid ORDER BY runcycleid DESC) as rn
      FROM
          factRequestDetails
      WHERE
          sourceoftruth = 'FOIMOD' AND activeflag = 'Y'
  ) AS rd ON rfd.foirequestid = rd.foirequestid AND rd.rn = 1
  LEFT JOIN dimRequestForDocumentsStatus rfds
    ON try_cast(rfd.reqfordocstatusid AS BIGINT) = rfds.reqfordocstatusid AND rfd.sourceoftruth = rfds.sourceoftruth
  -- LEFT JOIN dimRequestTypes rt
  --   ON rt.requesttypeid = rfd.requesttypeid
  LEFT JOIN dimECOffice eco
    ON eco.officecode = rd.officeid
  LEFT JOIN factRequestRFDCalculatedFields frfdcf
    ON rfd.foirequestid = frfdcf.foirequestid
    -- AND rfd.actionid = frfdcf.actionid
WHERE
  rfd.rn = 1

In [0]:
%sql

CREATE OR REPLACE TABLE default.materialized_viw_avgdaysinrequeststate AS
SELECT DISTINCT
  rsd.foiflowrequeststatusdetailsid,
  rsd.runcycleid,
  rsd.foirequestnumber,
  rsd.version,
  rsd.isactive,
  rsd.requeststatusid,
  rsd.ministrycode,
  rsd.assignedgroup,
  rsd.assignedministryperson,
  rsd.assignedministrygroup,
  rsd.startdate,
  rsd.duedate,
  rsd.cfrduedate,
  rsd.closedate,
  rsd.createdby,
  rsd.createddate,
  rsd.modifiedby,
  rsd.modifieddate,
  ffrs.name,
  LAG(ffrs.name) OVER (
      PARTITION BY rsd.foirequestnumber
      ORDER BY rsd.foirequestnumber, rsd.createddate, rsd.requeststatusid
    ) AS laststatusname
FROM
  factFOIFlowRequestStatusDetails rsd
    JOIN dimFOIFlowRequestStatus ffrs
      ON ffrs.requeststatusid = rsd.requeststatusid

In [0]:
%sql

-- currently join the most recent extension from foirequestextensions

CREATE OR REPLACE TABLE default.materialized_viw_oipc_10_1dreport AS
SELECT
  rd.visualrequestfilenumber AS Request_ID,
  rd.targetdate AS Due_Date,
  re.extendeddate AS Proposed_Date,
  rd.primaryusername AS Primary_User
FROM
  -- factRequestDetails rd
    (
      SELECT
          *,
          ROW_NUMBER() OVER(PARTITION BY foirequestid ORDER BY runcycleid DESC) as rn
      FROM
          factRequestDetails
      WHERE
          sourceoftruth = 'FOIMOD' AND activeflag = 'Y'
    ) AS rd
    -- LEFT JOIN dimRequests r ON rd.foirequestid = r.foirequestid AND rd.sourceoftruth = r.sourceoftruth
    -- JOIN factRequestExtensions re ON re.foirequestid = rd.foirequestid AND re.activeflag = 'Y'
    JOIN (
        SELECT
            *,
            ROW_NUMBER() OVER(PARTITION BY foirequestid ORDER BY runcycleid DESC) as rn
        FROM
            factRequestExtensions
        WHERE
            sourceoftruth = 'FOIMOD' AND activeflag in ('true', 't')
    ) AS re ON rd.foirequestid = re.foirequestid AND re.rn = 1
    LEFT JOIN dimExtensionTypes et ON et.extensiontypeid = re.extensiontypeid AND et.sourceoftruth = re.sourceoftruth
WHERE
  rd.rn = 1
  -- AND et.extensiontypename NOT LIKE 'PB%'
  -- AND re.cstatus = 'P'
  AND et.extensiontypename != 'Public Body'
ORDER BY
  rd.visualrequestfilenumber

In [0]:
%sql

CREATE OR REPLACE TABLE default.materialized_viw_scanningreport AS
SELECT
  rd.visualrequestfilenumber AS Request_ID,
--   rd.extension AS Number_of_Extensions_Applied,
  ext.numberofextensions AS Number_of_Extensions_Applied,
  frccf.physicalpageestimate AS Physical_Page_Estimate,
  frccf.electronicpageestimate AS Electronic_Page_Estimate,
  rd.remainingdays AS Remaining_Days,
  rd.targetdate AS Due_Date,
  rd.primaryusername AS Primary_User
FROM
  -- factRequestDetails rd
    (
      SELECT
          *,
          ROW_NUMBER() OVER(PARTITION BY foirequestid ORDER BY runcycleid DESC) as rn
      FROM
          factRequestDetails
      WHERE
          sourceoftruth = 'FOIMOD' AND activeflag = 'Y'
    ) AS rd
    LEFT JOIN dimRequests r ON rd.foirequestid = r.foirequestid and rd.sourceoftruth = r.sourceoftruth
    -- LEFT JOIN factRequestDocumentsDetails rdd ON rdd.foirequestid = rd.foirequestid AND rdd.activeflag = 'Y'
    LEFT JOIN (
        SELECT
            *,
            ROW_NUMBER() OVER(PARTITION BY foirequestid ORDER BY runcycleid DESC) as rn
        FROM
            factRequestDocumentsDetails
        WHERE
            sourceoftruth = 'FOIMOD' AND activeflag = 'Y'
    ) AS rdd ON rd.foirequestid = rdd.foirequestid AND rdd.rn = 1
    -- LEFT JOIN factRequestCustomCalcFields frccf ON rd.foirequestid = frccf.foirequestid
    LEFT JOIN (
        SELECT
            *,
            ROW_NUMBER() OVER(PARTITION BY foirequestid ORDER BY runcycleid DESC) as rn
        FROM
            factRequestCustomCalcFields
        WHERE
            sourceoftruth = 'FOIMOD'
    ) AS frccf ON rd.foirequestid = frccf.foirequestid AND frccf.rn = 1
    LEFT JOIN (
        SELECT
            foirequestid,
            COUNT(DISTINCT requestextid) AS numberofextensions
        FROM
            factRequestExtensions
        WHERE
            sourceoftruth = 'FOIMOD'
            AND approvedstatus = 'Approved'
            AND activeflag = in ('true', 't')
        GROUP BY
            foirequestid
    ) AS ext ON rd.foirequestid = ext.foirequestid
WHERE
  rd.rn = 1

In [0]:
%sql

-- --optimize factRequestDivisionalStages after ETL
-- OPTIMIZE factRequestDivisionalStages
-- ZORDER BY (foirequestnumber);

CREATE OR REPLACE TABLE default.materialized_viw_analystworkloaddash AS
SELECT DISTINCT
  rd.foirequestid,
  rd.visualrequestfilenumber AS requestid,
  CASE
    WHEN ecg.groupname = 'Consolidated Intake' THEN 'Intake'
    ELSE ecggn.groupname
  END AS team,
  CASE
    WHEN ecg.groupname = 'Consolidated Intake' THEN ecg.groupdescription
    ELSE ecggn.groupdescription
  END AS manager,
  rd.ministry,
  CASE
    WHEN (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') THEN eco.officecode
    ELSE rd.officeid
  END AS procorg,
  rd.applicantcategoryid AS category,
  rt.requesttypename AS type,
  rqt.requestertypename AS applicanttype,
  concat(req.lastname, ', ', req.firstname) AS applicantname,
  frccf.subject,
  rd.startdate,
  rd.closeddate AS enddate,
  rd.targetdate AS duedate,
  rd.processeddays AS totalprocesseddays,
  rd.currentactivity,
  frccf.currentactivitydate,
  rd.primaryusername AS analyst,
  frccf.crossgovtno,
  rd.requestdescription AS description,
  rd.status,
  frccf.customfieldstatus AS CustomStatus,
  frccf.countontime,
  frccf.countoverdue,
  frccf.onholddays,
  frccf.daysoverdue,
  CASE
    WHEN rd.requeststatus = 'Closed' THEN 'Closed'
    ELSE 'Not Closed'
  END AS notclosed,
  rpt.estimatedamount AS feesest,
  rpt.incurredamount AS feeswaived,
  rpt.paidamount AS feespaid,
  rd.disposition,
  frccf.publication,
  frccf.publicationreason,
  CASE
    WHEN rd.extension IS NULL THEN 'N'
    ELSE 'Y'
  END AS extension,
  rd.execcomments,
  frccf.noofpagesreleased AS noofpagesdelivered,
  rdd.noofpagesintherequest,
  CASE
    WHEN
      month(rd.receiveddate) BETWEEN 4 AND 12
    THEN
      concat(year(rd.receiveddate), year(add_months(rd.receiveddate, 12)))
    WHEN month(rd.receiveddate) IS NULL THEN ''
    ELSE concat(year(add_months(rd.receiveddate, -12)), year(rd.receiveddate))
  END AS startfyr,
  CASE
    WHEN
      month(rd.closeddate) BETWEEN 4 AND 12
    THEN
      concat(year(rd.closeddate), year(add_months(rd.closeddate, 12)))
    WHEN month(rd.closeddate) IS NULL THEN ''
    ELSE concat(year(add_months(rd.closeddate, -12)), year(rd.closeddate))
  END AS endfyr,
  CASE
    WHEN month(rd.receiveddate) BETWEEN 4 AND 6 THEN 1
    WHEN month(rd.receiveddate) BETWEEN 7 AND 9 THEN 2
    WHEN month(rd.receiveddate) BETWEEN 10 AND 12 THEN 3
    WHEN month(rd.receiveddate) BETWEEN 1 AND 3 THEN 4
    ELSE NULL
  END AS startfqtr,
  CASE
    WHEN month(rd.closeddate) BETWEEN 4 AND 6 THEN 1
    WHEN month(rd.closeddate) BETWEEN 7 AND 9 THEN 2
    WHEN month(rd.closeddate) BETWEEN 10 AND 12 THEN 3
    WHEN month(rd.closeddate) BETWEEN 1 AND 3 THEN 4
    ELSE NULL
  END AS endfqtr,
  d.division,
  s.name AS stage,
  CASE
    WHEN rd.requeststatus = 'Closed' THEN 'Closed'
    WHEN frccf.countontime = 1 THEN 'Active'
    WHEN frccf.countoverdue = 1 THEN 'Overdue'
    ELSE NULL
  END AS overduestatus,
  ffrs.name AS foiflowstatus,
  ffrsd.ministryassigneefullname AS assignedministryperson,
  rds2.divisions,
  rd.requeststatus,
  month(rd.startdate) AS startmonth,
  rd.requesttypeid,
  COALESCE(try_cast(ffrsd.recordspagecount AS INT), 0) AS recordspagecount,
  COALESCE(try_cast(ffrsd.lanpagecount AS INT), 0) AS lanpagecount
FROM
    (
      SELECT
          *,
          ROW_NUMBER() OVER(PARTITION BY foirequestid ORDER BY runcycleid DESC) as rn
      FROM
          factRequestDetails
      WHERE
          sourceoftruth = 'FOIMOD' AND activeflag = 'Y'
    ) AS rd
    -- factRequestDetails AS rd
    LEFT JOIN dimECGroups ecg
      ON rd.primarygroupid = ecg.groupid and ecg.sourceoftruth = rd.sourceoftruth
    LEFT JOIN dimECGroups ecggn
      ON rd.groupname = ecggn.groupname and ecggn.sourceoftruth = rd.sourceoftruth
    LEFT JOIN
        dimECOffice AS eco ON CASE
            WHEN (rd.sourceoftruth IS NULL OR rd.sourceoftruth != 'FOIMOD') THEN eco.officeid = rd.officeid
            ELSE FALSE
        END
    LEFT JOIN dimRequestTypes rt
      ON rd.requesttypeid = rt.requesttypeid
    -- LEFT JOIN factRequestPaymentTransaction rpt
    --   ON rd.foirequestid = rpt.foirequestid
    --   AND rpt.activeflag = 'Y'
    LEFT JOIN (
        SELECT
            *,
            ROW_NUMBER() OVER(PARTITION BY foirequestid ORDER BY runcycleid DESC) as rn
        FROM
            factRequestPaymentTransaction
        WHERE
            sourceoftruth = 'FOIMOD' AND activeflag = 'Y'
    ) AS rpt ON rd.foirequestid = rpt.foirequestid AND rpt.rn = 1
    -- LEFT JOIN factRequestDocumentsDetails rdd
    --   ON rd.foirequestid = rdd.foirequestid
    --   AND rdd.activeflag = 'Y'
    LEFT JOIN (
        SELECT
            *,
            ROW_NUMBER() OVER(PARTITION BY foirequestid ORDER BY runcycleid DESC) as rn
        FROM
            factRequestDocumentsDetails
        WHERE
            sourceoftruth = 'FOIMOD' AND activeflag = 'Y'
    ) AS rdd ON rd.foirequestid = rdd.foirequestid AND rdd.rn = 1
    -- LEFT JOIN factRequestRequesters rr
    --   ON rd.foirequestid = rr.foirequestid
    --   AND rr.activeflag = 'Y'
    LEFT JOIN (
        SELECT
            *,
            ROW_NUMBER() OVER(PARTITION BY foirequestid ORDER BY runcycleid DESC) as rn
        FROM
            factRequestRequesters
        WHERE
            sourceoftruth = 'FOIMOD' AND activeflag = 'Y'
    ) AS rr ON rd.foirequestid = rr.foirequestid AND rr.rn = 1
    LEFT JOIN dimRequesterTypes rqt
      ON rd.applicantcategoryid = rqt.requestertypeid and rd.sourceoftruth = rqt.sourceoftruth
    LEFT JOIN dimRequesters req
      ON req.sourceoftruth = 'FOIMOD' AND req.foirequestid = rd.foirequestid AND req.requesterid = rd.requesterid
    -- LEFT JOIN factRequestCustomCalcFields frccf
    --   ON rd.foirequestid = frccf.foirequestid and rd.sourceoftruth = frccf.sourceoftruth
    LEFT JOIN (
        SELECT
            *,
            ROW_NUMBER() OVER(PARTITION BY foirequestid ORDER BY runcycleid DESC) as rn
        FROM
            factRequestCustomCalcFields
        WHERE
            sourceoftruth = 'FOIMOD'
    ) AS frccf ON rd.foirequestid = frccf.foirequestid AND frccf.rn = 1
    -- LEFT JOIN (
    --   SELECT
    --     foirequestnumber,
    --     MAX(runcycleid) AS runcycleid,
    --     MAX(createddate) AS createddate
    --   FROM
    --     factRequestDivisionalStages
    --   GROUP BY
    --     foirequestnumber
    -- ) urds
    --   ON urds.foirequestnumber = rd.visualrequestfilenumber
    -- LEFT JOIN factRequestDivisionalStages rds
    --   ON rds.foirequestnumber = rd.visualrequestfilenumber
    --   AND urds.runcycleid = rds.runcycleid
    --   AND urds.createddate = rds.createddate
    LEFT JOIN (
        SELECT
            *,
            ROW_NUMBER() OVER(PARTITION BY foirequestnumber ORDER BY runcycleid DESC, createddate DESC) AS rn
        FROM
            factRequestDivisionalStages
    ) rds
    ON rds.foirequestnumber = rd.visualrequestfilenumber AND rds.rn = 1
    LEFT JOIN dimStages s
      ON s.stageid = rds.stageid and s.sourceoftruth = rd.sourceoftruth
    LEFT JOIN dimDivisions d
      ON d.divisionid = rds.divisionid and rd.sourceoftruth = d.sourceoftruth
    -- LEFT JOIN (
    --   SELECT
    --     foirequestnumber,
    --     MAX(runcycleid) AS runcycleid,
    --     MAX(createddate) AS createddate
    --   FROM
    --     factFOIFlowRequestStatusDetails
    --   GROUP BY
    --     foirequestnumber
    -- ) uffrsd
    --   ON uffrsd.foirequestnumber = rd.visualrequestfilenumber
    -- LEFT JOIN factFOIFlowRequestStatusDetails ffrsd
    --   ON ffrsd.foirequestnumber = rd.visualrequestfilenumber
    --   AND uffrsd.runcycleid = ffrsd.runcycleid
    --   AND uffrsd.createddate = ffrsd.createddate
    LEFT JOIN (
        SELECT
            *,
            ROW_NUMBER() OVER(PARTITION BY foirequestnumber ORDER BY runcycleid DESC, createddate DESC) AS rn
        FROM
            factFOIFlowRequestStatusDetails
    ) ffrsd
    ON ffrsd.foirequestnumber = rd.visualrequestfilenumber AND ffrsd.rn = 1
    LEFT JOIN dimFOIFlowRequestStatus ffrs
      ON ffrs.requeststatusid = ffrsd.requeststatusid
    LEFT JOIN (
      SELECT
        rds_1.foirequestnumber,
        string_agg(DISTINCT CONCAT('<p>', d_1.division, ' - ', s_1.name, '</p>'), '\n') AS divisions
      FROM
        -- (
        --   SELECT
        --     foirequestnumber,
        --     MAX(foirequestversionid) AS foirequestversionid,
        --     MAX(runcycleid) AS runcycleid
        --   FROM
        --     factRequestDivisionalStages
        --   GROUP BY
        --     foirequestnumber
        -- ) sq
        --   JOIN factRequestDivisionalStages rds_1
        --     ON rds_1.foirequestnumber = sq.foirequestnumber
        --     AND rds_1.foirequestversionid = sq.foirequestversionid
        --     AND rds_1.runcycleid = sq.runcycleid
          (
              SELECT
                  *,
                  ROW_NUMBER() OVER(PARTITION BY foirequestnumber ORDER BY foirequestversionid DESC, runcycleid DESC) AS rn
              FROM
                  factRequestDivisionalStages
          ) rds_1
          JOIN dimStages s_1
            ON s_1.stageid = rds_1.stageid
          JOIN dimDivisions d_1
            ON d_1.divisionid = rds_1.divisionid
          WHERE rds_1.rn = 1
      GROUP BY
        rds_1.foirequestnumber
    ) rds2
      ON rds2.foirequestnumber = rd.visualrequestfilenumber
WHERE
  rd.rn = 1
  -- AND rd.officeid NOT IN ("1", "10", "21", "33", "41", "45", "49", "51", "55", 'CLB', 'IIO', 'MGC', 'OBC', 'TIC')
  AND rd.requesttypeid IN (31, 33);


In [0]:
%sql

CREATE OR REPLACE TABLE default.materialized_viw_eccrequeststate AS
WITH RequestStatusData AS (
    SELECT
        t.foirequestnumber,
        t.ministrycode,
        t.name,
        t.laststatusname,
        t.startdate,
        t.closedate,
        t.createddate,
        -- Calculate the end date for the current status by looking at the createddate of the next status
        LEAD(t.createddate) OVER (
            PARTITION BY t.foirequestnumber
            ORDER BY t.createddate, t.foirequestnumber, t.name
        ) AS statusenddate
    FROM
        viw_avgdaysinrequeststate t
    -- The original query's innermost WHERE clause: exclude records where name equals laststatusname
    WHERE
        t.name <> t.laststatusname
        OR t.laststatusname IS NULL
)
SELECT
    r.foirequestnumber,
    r.ministrycode,
    r.name,
    r.laststatusname,
    -- Calculate 'days' only for statuses that have an end date
    (r.statusenddate - r.createddate) AS days,
    r.startdate,
    r.closedate
FROM
    RequestStatusData r
-- Apply the outer WHERE clauses
WHERE
    r.statusenddate IS NOT NULL  -- Original second-to-last WHERE
    AND r.ministrycode = 'ECC'  -- Original outermost WHERE
ORDER BY
    r.foirequestnumber,
    r.name,
    r.laststatusname;